# Обратная задача ЭЭГ

Обратная задача ЭЭГ заключается в восстановлении распределения источников (тока и заряда) внутри области (например, головы) на основе измерений электрического потенциала на границе.

В данном ноутбуке мы используем нейросеть для моделирования распределения заряда и тока, а также обновляем функцию потерь и пайплайн для решения этой задачи.

In [ ]:
# Импорт необходимых библиотек
using NeuralPDE, Lux, ModelingToolkit, Optimization, OptimizationOptimJL, LineSearches,
      OptimizationOptimisers, LuxCUDA, Random, ComponentArrays
using ModelingToolkit: Interval, infimum, supremum
using Distributions, Plots

In [ ]:
const gpud = gpu_device()
const cpud = cpu_device()

(::CPUDevice) (generic function with 4 methods)

In [ ]:
# Определение прямой задачи
@parameters x, y, z, t
@variables φ(..), Ax(..),Ay(..),Az(..), ρ(..), jx(..), jy(..), jz(..)
A = [Ax, Ay, Az]
j = [jx, jy, jz]
Dxx = Differential(x)^2
Dyy = Differential(y)^2
Dzz = Differential(z)^2

# Определение физических постоянных
const ε₀ = 8.854187817e-12 # Электрическая постоянная в вакууме (Ф/м)
const ε = ε₀ # Электрическая постоянная (Ф/м)
const μ₀ = 1.2566370614e-6 # Магнитная постоянная в вакууме (Гн/м)
const μ = μ₀ # Магнитная постоянная (Гн/м)
const c = 299792458.0 # Скорость света в вакууме (м/с)

# Определение оператора Лапласа как функции
function laplacian(F, params)
    return sum((Differential(param)^2)(F) for param in params)
end

# Определение оператора Даламбера как функции
function dalembert_operator(F, params, ε, μ, c)
    Δ = laplacian(F, params)
    return Δ  - (ε * μ / c^2) * (Differential(t)^2)(F)
end


dalembert_operator (generic function with 1 method)

In [ ]:
dalembert_operator(φ(x,y,z,t), [x, y, z], ε, μ, c)

Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t)))

In [ ]:
# Уравнение
eqs = [
    [dalembert_operator(φ(x, y, z, t), [x, y, z], ε, μ, c) ~ -ρ(x, y, z, t) / (ε * ε₀)];
    [dalembert_operator(A[i](x, y, z, t), [x, y, z], ε, μ, c) ~ -μ * μ₀ * j[i](x, y, z, t) for i in 1:3]
]

4-element Vector{Equation}:
 Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -1.27556484025426e22ρ(x, y, z, t)
 Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -1.5791367040840272e-12jx(x, y, z, t)
 Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -1.5791367040840272e-12jy(x, y, z, t)
 Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Az(x, y, z, t))) + Different

In [ ]:
# Границы области
const x_min, x_max = -10.0, 10.0
const y_min, y_max = -10.0, 10.0
const z_min, z_max = -10.0, 10.0
const t_min, t_max = 0.0, 1.0
# Область
domains = [x ∈ Interval(x_min, x_max),
           y ∈ Interval(y_min, y_max),
           z ∈ Interval(z_min, z_max), 
           t ∈ Interval(t_min, t_max)]
# Граничные условия
measured_points = [
    ((0.5, 0.5, 0.5, 0), 1.0),
    ((0.2, 0.3, 0.4, 0), 0.0),
    ((0.8, 0.7, 0.6, 0),  -1.0)
]

bcs = [
    [φ(x_min, y, z, t) ~ 0.0, φ(x_max, y, z, t) ~ 0.0,
    φ(x, y_min, z, t) ~ 0.0, φ(x, y_max, z, t) ~ 0.0,
    φ(x, y, z_min, t) ~ 0.0, φ(x, y, z_max, t) ~ 0.0];
    [A[i](x_min, y, z, t)  ~ 0.0 for i in 1:3]
]


12-element Vector{Equation}:
 φ(-10.0, y, z, t) ~ 0.0
 φ(10.0, y, z, t) ~ 0.0
 φ(x, -10.0, z, t) ~ 0.0
 φ(x, 10.0, z, t) ~ 0.0
 φ(x, y, -10.0, t) ~ 0.0
 φ(x, y, 10.0, t) ~ 0.0
 Ax(-10.0, y, z, t) ~ 0.0
 Ay(-10.0, y, z, t) ~ 0.0
 Az(-10.0, y, z, t) ~ 0.0
 φ(0.5, 0.5, 0.5, 0) ~ 1.0
 φ(0.2, 0.3, 0.4, 0) ~ 0.0
 φ(0.8, 0.7, 0.6, 0) ~ -1.0

In [ ]:
function additional_loss_weightened(lambda)
    function additional_loss(phi_pred_fun, θ)
        return lambda * sum(abs2(phi_pred_fun(x, y, z, t, θ) - phi) for ((x, y, z, t, θ), phi) in measured_points) / length(measured_points)
    end
    return additional_loss
end

additional_loss_weightened (generic function with 1 method)

In [ ]:

# Нейросеть
# Определение новой нейросети для моделирования распределения заряда и тока
input_ = 4  # x, y, z
n = 32      # число нейронов в скрытых слоях
# Функция для разделения выхода сети на переменные
"""
function split_outputs(out)
    φ_pred = out[1]
    A_pred = out[2:4]
    ρ_pred = out[5]
    j_pred = out[6:8]
    return φ_pred, A_pred, ρ_pred, j_pred
end

chain = [Chain(
    Dense(input_, n, σ),
    Dense(n, n, σ),
    Dense(n, 1)
) for _ in 1:8] 

# Определение системы
ps = [Lux.setup(Random.default_rng(), chain[i])[1] |> ComponentArray |> gpud .|> Float64 for i in 1:8]
"""
chain = Chain(
    Dense(input_, n, σ),
    Dense(n, n, σ),
    Dense(n, n, σ),
    Dense(n, 8)
)

# Определение системы
ps = Lux.setup(Random.default_rng(), chain)[1] |> ComponentArray |> gpud .|> Float64

strategy = QuasiRandomTraining(4096)
discretization = PhysicsInformedNN(chain, strategy; init_params = ps, additional_loss = additional_loss_weightened(0.2))

PhysicsInformedNN{Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, QuasiRandomTraining{QuasiMonteCarlo.LatinHypercubeSample}, ComponentVector{Float64, CuArray{Float64, 1, CUDA.DeviceMemory}, Tuple{Axis{(layer_1 = ViewAxis(1:160, Axis(weight = ViewAxis(1:128, ShapedAxis((32, 4))), bias = ViewAxis(129:160, Shaped1DAxis((32,))))), layer_2 = ViewAxis(161:1216, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_3 = ViewAxis(1217:2272, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_4 = ViewAxis(2273:2536, Axis(weight = ViewAxis(1:256, ShapedAxis((8, 32))), bias = ViewAxis(257:264, Shaped1DAxis((8,)))

In [ ]:
allvars = [φ(x, y, z, t); [A_(x, y, z, t) for A_ in A]; ρ(x, y, z, t); [j_(x, y, z, t) for j_ in j]]

8-element Vector{Num}:
  φ(x, y, z, t)
 Ax(x, y, z, t)
 Ay(x, y, z, t)
 Az(x, y, z, t)
  ρ(x, y, z, t)
 jx(x, y, z, t)
 jy(x, y, z, t)
 jz(x, y, z, t)

In [ ]:
@named pde_system = PDESystem(eqs, bcs, domains, [x, y, z, t], allvars)

PDESystem
Equations: Equation[Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -1.27556484025426e22ρ(x, y, z, t), Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -1.5791367040840272e-12jx(x, y, z, t), Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -1.5791367040840272e-12jy(x, y, z, t), Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Az(x, y, z, t))) + Differen

In [ ]:
prob = discretize(pde_system, discretization)
sym_prob = symbolic_discretize(pde_system, discretization)

NeuralPDE.PINNRepresentation(Equation[Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -1.27556484025426e22ρ(x, y, z, t), Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -1.5791367040840272e-12jx(x, y, z, t), Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -1.5791367040840272e-12jy(x, y, z, t), Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.2379901471139933e-34Differential(t)(Differential(t)(Az(x, y, z, t))) + 

In [ ]:
pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    return false
end

#13 (generic function with 1 method)

In [ ]:
# Оптимизация
opt = OptimizationOptimisers.Adam(0.01)

Adam(0.01, (0.9, 0.999), 1.0e-8)

In [ ]:

# Решение
res = solve(prob, opt; maxiters = 800, callback, log_frequency = 50)
phi = discretization.phi

loss: 4.024886251979774e43
pde_losses: [4.025020562575494e43, 5.6009600936029174e-6, 5.733427505257786e-6, 5.6484676277153615e-6]
bcs_losses: [0.251062689842218, 0.24394682346423113, 0.2554834235095773, 0.23974781742980156, 0.2432677703734727, 0.2510249410183376, 0.2510555644320924, 0.25101565940565396, 0.2509814971880835, 0.9769096582995062, 0.24820225852728864, 1.5169892307817319]
loss: 1.3955180412244277e43
pde_losses: [1.3961030161793641e43, 5.5798397864884894e-6, 5.549310916626425e-6, 5.544727151750137e-6]
bcs_losses: [0.08649250353215895, 0.08492788334675747, 0.09070473808633045, 0.08134360462062673, 0.08510908584907098, 0.08622714532303652, 0.08647719602488878, 0.08646429368163902, 0.08644195831745372, 0.9445412799252517, 0.08563853013848145, 1.225573940915099]
loss: 3.159434331876468e42
pde_losses: [3.156448437105992e42, 5.164011393871908e-6, 5.2233375437172946e-6, 5.280756116591959e-6]
bcs_losses: [0.019009597858451195, 0.019389877199159647, 0.021032191824273197, 0.01809792982

NeuralPDE.Phi{StatefulLuxLayer{Static.True, Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, Nothing, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}, layer_4::@NamedTuple{}}}}(StatefulLuxLayer{Static.True, Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, Nothing, @NamedTuple{layer_1::@NamedTuple{}, layer_2::@NamedTuple{}, layer_3::@NamedTuple{}, layer_4::@NamedTuple{}}}(Chain{@NamedTuple{layer_1::Dense{typeof(σ